<img src="https://github.com/GodStorm91/mobydock/blob/master/mobydock/static/docker-logo.png?raw=true" width="10%"> ➕
<img src="https://github.com/Microsoft/azuredatastudio/blob/master/samples/notebookSamples/Graphics/AzureDataStudioLogo.png?raw=true" width="6.5%"> = ❤

# This PowerShell Notebook will walk you through how to setup a SQL-on-Linux instance in a Docker container, on your local [Windows] machine.

### Prerequisite Steps
0) Before any of this can work, you must have Docker Destop running. 
    You must also have the latest SqlServer module installed from the PowerShell Gallery.
`Install-Module SqlServer`

In [0]:
Install-Module SqlServer;
Import-Module SqlServer;

0B) Use the code to download the AdventureWorks2016.bak file from GitHub

In [35]:
$BakURL = "https://github.com/Microsoft/sql-server-samples/releases/download/adventureworks/AdventureWorks2016.bak";
$BakFile = "$($Home)/Downloads/AdventureWorks2016.bak";
Invoke-WebRequest -Uri $BakURL -OutFile $BakFile;

$BakURL = "https://github.com/Microsoft/sql-server-samples/releases/download/adventureworks/AdventureWorks2016.bak";
$BakFile = "$($Home)/Downloads/AdventureWorks2016.bak";
Invoke-WebRequest -Uri $BakURL -OutFile $BakFile;


0C) Create a local directory for the container to use to store database files, outside of the container.

In [3]:
mkdir C:/SQLData/Docker/SQLDev01



    Directory: C:\SQLData\Docker


Mode                LastWriteTime         Length Name                                                                  
----                -------------         ------ ----                                                                  
d-----        7/29/2019   2:58 PM                SQLDev67                                                              




## Create your SQL-on-Linux Docker Container
1) Create a SQL-on-Linux Docker Container with just the code below.
    NOTE: You may want to change the password used for the SA account.
    You may also want to change the Volumne path: "C:\SQLData\Docker\SQLDev01"

In [6]:
docker run -d -p 10001:1433 -e ACCEPT_EULA=Y -e SA_PASSWORD=Test1234 --name testcontainer01 mcr.microsoft.com/mssql/server:2019-latest

571715f109792189a0fcba1fa0ac70e481d91573a99b888218329cb436853e3c


Verify that the SQL instance in the container is up & running

In [8]:
docker ps

CONTAINER ID        IMAGE                                        COMMAND                  CREATED             STATUS              PORTS                     NAMES
571715f10979        mcr.microsoft.com/mssql/server:2019-latest   "/opt/mssql/bin/perm…"   30 seconds ago      Up 30 seconds       0.0.0.0:10001->1433/tcp   testcontainer01
41f9a28848ab        mcr.microsoft.com/mssql/server:2019-latest   "/opt/mssql/bin/perm…"   13 hours ago        Up 13 hours         0.0.0.0:10003->1433/tcp   testcontainer03


In [2]:
$SqlCred = (Get-Credential sa)

Verify that the SQL instance in the container is up & running

In [10]:
Get-SqlInstance -ServerInstance 'localhost,10001' -Credential $SqlCred


Instance Name                   Version    ProductLevel UpdateLevel  HostPlatform HostDistribution                
-------------                   -------    ------------ -----------  ------------ ----------------                
localhost,10001                 15.0.2070  RTM          n/a          Linux        Ubuntu                          




Now verify the container only has the system databases.

In [12]:
Get-SqlDatabase -ServerInstance 'localhost,10001' -Credential $SqlCred


Name                 Status           Size     Space  Recovery Compat. Owner                     Collation             
                                            Available  Model     Level                                                 
----                 ------           ---- ---------- -------- ------- -----                     ---------             
master               Normal        6.44 MB 1008.00 KB Simple       150 sa                        SQL_Latin1_General_CP1
                                                                                                 _CI_AS                
model                Normal       16.00 MB    5.70 MB Full         150 sa                        SQL_Latin1_General_CP1
                                                                                                 _CI_AS                
msdb                 Normal       13.88 MB    1.41 MB Simple       150 sa                        SQL_Latin1_General_CP1
                                       

2) Copy the backup file to the directory your container volume is mapped to.
    Make sure you use the location you stored the .bak file in.

The next command is mandatoru for placing the backup file in a location accessible to the `Restore-SqlDatabase` command below.


In [16]:
docker cp "$($Home)/Downloads/AdventureWorks2016.bak" testcontainer01:'var/opt/mssql/data/'

This next step is optional, just to allow you to verify the .BAK file has been copied to the correct location.

These next two commands should be run inside the Azure Data Studio terminal, **not within the PowerShell Notebook** itself.

```
docker exec -it testcontainer01 bash
```
Once inside of bash, run this simple command to search for the .BAK file:
```
ls var/opt/mssql/data/
```

3) Restore the AdventureWorks2016 databas

In [20]:
Restore-SqlDatabase -ServerInstance 'localhost,10001' -Credential $SqlCred -BackupFile '/var/opt/mssql/data/AdventureWorks2016.bak' -Database 'AdventureWorks2016' -AutoRelocateFile

In [22]:
Get-SqlDatabase -ServerInstance 'localhost,10001' -Credential $SqlCred


Name                 Status           Size     Space  Recovery Compat. Owner                     Collation             
                                            Available  Model     Level                                                 
----                 ------           ---- ---------- -------- ------- -----                     ---------             
AdventureWorks2016   Normal      209.63 MB    1.31 MB Simple       130 sa                        SQL_Latin1_General_CP1
                                                                                                 _CI_AS                
master               Normal        6.44 MB 1008.00 KB Simple       150 sa                        SQL_Latin1_General_CP1
                                                                                                 _CI_AS                
model                Normal       16.00 MB    5.70 MB Full         150 sa                        SQL_Latin1_General_CP1
                                       